In [155]:
import numpy as np
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense 
from keras.layers import ConvLSTM2D, Conv2D, MaxPooling2D
from keras.layers import MaxPooling1D, Conv1D
from keras.layers import GRU, LSTM, BatchNormalization
from keras.layers import TimeDistributed
from keras.layers import Flatten
from keras.utils import to_categorical
from keras.layers import Activation

A01T = h5py.File('datasets/A01T_slice.mat','r')
data = np.copy(A01T['image'])
labels = np.copy(A01T['type'])
labels = labels[0,0:data.shape[0]:1]
labels = np.asarray(labels, dtype=np.int32)

a = data[:56]
b = data[57:]
data = np.vstack((a,b))
a = labels[:56]
b = labels[57:]
labels = np.hstack((a,b))
#enc = OneHotEncoder()
#enc_labels = enc.fit_transform(labels.reshape(-1,1)).toarray()
enc_labels = to_categorical(labels-769, num_classes=4)
print(enc_labels)

[[ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 ..., 
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]


In [175]:
data2d = data.reshape(data.shape[0], data.shape[2], data.shape[1], 1)
data1d = data.reshape(data.shape[0], data.shape[2], 25)
print data2d.shape
print data1d.shape

(287, 1000, 25, 1)
(287, 1000, 25)


In [188]:
bs, t, f = data.shape
np.random.seed(42)
shuffle = np.random.choice(bs,bs,replace=False)
""" 
train_samples = 200
train_data = newdata[shuffle[:train_samples],:,:]
train_labels = enc_labels[shuffle[:train_samples]]
test_data = newdata[shuffle[train_samples:],:,:]
test_labels = enc_labels[shuffle[train_samples:]]

"""
#mask = np.ones_like(data)
#mask[[:,:,21:24]] = True
#newdata = data[mask]
#np.delete(mask, :,:,24)
train_samples = 200
train_data = data2d[shuffle[:train_samples],:,:]
train_labels = enc_labels[shuffle[:train_samples]]
test_data = data2d[shuffle[train_samples:],:,:]
test_labels = enc_labels[shuffle[train_samples:]]
print data2d.shape
print train_data.shape

(287, 1000, 25, 1)
(200, 1000, 25, 1)


In [ ]:
# Conv2D + LSTM
"""
cnn = Sequential()
cnn.add(ConvLSTM2D(1, (2,2), strides=2, activation='relu',input_shape=(1000, 5, 5, 1)))
cnn.add(Flatten())
cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = cnn.fit(train_data,train_labels,epochs=5,validation_split=0.25,batch_size=32)
print(model.summary())
"""


model = Sequential()
model.add(Conv2D(40, kernel_size=(25,1), activation='relu', data_format='channels_last', input_shape=(1000, 25, 1)))  
model.add(Conv2D(40, kernel_size=(1,25), activation='relu') ) 
model.add(MaxPooling2D(pool_size=(60,1)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])
model.summary()
hist = model.fit(train_data,train_labels,epochs=10,validation_split=0.25,batch_size=32)
#test_score = model.evaluate(test_data, test_labels, batch_size=32)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_126 (Conv2D)          (None, 976, 25, 40)       1040      
_________________________________________________________________
conv2d_127 (Conv2D)          (None, 976, 1, 40)        40040     
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 16, 1, 40)         0         
_________________________________________________________________
flatten_46 (Flatten)         (None, 640)               0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 640)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 4)                 2564      
Total params: 43,644
Trainable params: 43,644
Non-trainable params: 0
_________________________________________________________________
Train 